<a href="https://colab.research.google.com/github/Michael-Jimenez-C/Investigaci-n-de-operaciones/blob/main/Tarea2IO/Metodo-Grafico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

#Funcion

In [ ]:
class Funcion:
  MENOR_IGUAL=0
  MAYOR_IGUAL=1
  MAYOR=2
  MENOR=3
  def __init__(self,coeficiente:float,corte:float, tipo:int=MENOR):
    self.coeficiente=coeficiente
    self.corte=corte
    self.tipo=tipo
    self.funcion=None

  def __add__(self,other):
    '''
    Estas operaciones no se supone deban usarse por el usuario, son solo para poder hacer calculos sobre las intersecciones entre dos funciones.
    '''
    return Funcion(self.coeficiente+other.coeficiente,self.corte+other.corte,self.tipo)

  def __sub__(self,other):
    '''
    Estas operaciones no se supone deban usarse por el usuario, son solo para poder hacer calculos sobre las intersecciones entre dos funciones.
    '''
    return Funcion(self.coeficiente-other.coeficiente,self.corte-other.corte,self.tipo)

  def __repr__(self):
    return "{}x+({}) {} y".format(self.coeficiente,self.corte, ["<=",">=",">","<"][self.tipo]) 

  def ecuacion(coef1,coef2,cort1,cort2, tipo=MENOR):
    corte=(cort2-cort1)/coef1
    coeficiente=coef2/coef1
    if coef1<0:
      if tipo==Funcion.MENOR or tipo ==Funcion.MAYOR:
        tipo=(Funcion.MAYOR if tipo==Funcion.MENOR else Funcion.MENOR)
      else:
        tipo=(Funcion.MAYOR_IGUAL if tipo==Funcion.MENOR_IGUAL else Funcion.MENOR_IGUAL)
    return Funcion(coeficiente,corte,tipo)

  def getCorte(self):
    return ((0,self.corte),((-self.corte)/self.coeficiente,0))

  def evaluar(self,x):
    if self.funcion==None:
      self.funcion=lambda x:self.coeficiente*x+self.corte
    return self.funcion(x)

  def evaluarCondicion(self,x,y):
    return [lambda a,b: a<=b,lambda a,b: a>=b, lambda a,b:a>b, lambda a,b: a<b][self.tipo](round(y,5),round(self.evaluar(x),5))

  def getInterseccion(self, other):
    p=(self-other).getCorte()[1][0]
    return p,self.evaluar(p)


#Solver

In [ ]:
class Solver:
  def __init__(self, condiciones,positivas=False):
    self.condiciones=condiciones
    self.positivas=positivas
  def __puntos(self):
    k=self.condiciones
    p=[]
    for i in range(len(k)):
      for j in range(i+1,len(k)):
        p.append(k[i].getInterseccion(k[j]))
    if self.positivas:
      for i in range(len(k)):
          w=k[i].getCorte()
          p.append(w[0])
          p.append(w[1])
    return p

  def solver(self):
    p=self.__puntos()
    s=p[:]
    for i in self.condiciones:
      for j in p:
        if (not i.evaluarCondicion(j[0],j[1])) or (j[1]<0 or j[0]<0 if self.positivas else False):
          if j in s:
            s.remove(j)
    return s

In [ ]:
condiciones=[Funcion(-2,10,Funcion.MENOR_IGUAL),Funcion(10,-10,Funcion.MAYOR_IGUAL)]

w=Solver(condiciones,positivas=True)
w.solver()
x=np.linspace(-10,10,10)
for i in condiciones:
  plt.plot(x,i.evaluar(x))

y=np.array(w.solver()).T
if len(y)>0:
  plt.scatter(y[0],y[1],color="red")
plt.grid()
plt.show()